In [1]:
import cv2


### for code both  gray frame and frame
- this code will give blue color rectangle of thickness 10 , A video running stream.

In [3]:
capture = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")


while True:
    return_value, frame = capture.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if return_value == False :
        continue
    
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
# 1.3 is scale factor - how much the image size is reduced at each image scale
# 1.3 means image shrinks by 30%
# 1.05 means image shrinks by 5 %
# 5 is no. of neighbours-->> no. of neighbours each rectangles should have



    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 255, 0), 10)
#because in opencv we can make rectangle using two diagnally opposite points
#(255,0,0 ) is color of boundary , 10 is thickness of  line(blue color, here)

    cv2.imshow("Video Frame", frame)

    key_pressed_user =cv2.waitKey(1) & 0xFF 
    
    if key_pressed_user == ord('q'): 
        break

cap.release()
cv2.destroyAllWindows()


NameError: name 'cap' is not defined

In [4]:
cv2.rectangle?

Docstring:
rectangle(img, pt1, pt2, color[, thickness[, lineType[, shift]]]) -> img
.   @brief Draws a simple, thick, or filled up-right rectangle.
.   
.   The function cv::rectangle draws a rectangle outline or a filled rectangle whose two opposite corners
.   are pt1 and pt2.
.   
.   @param img Image.
.   @param pt1 Vertex of the rectangle.
.   @param pt2 Vertex of the rectangle opposite to pt1 .
.   @param color Rectangle color or brightness (grayscale image).
.   @param thickness Thickness of lines that make up the rectangle. Negative values, like #FILLED,
.   mean that the function has to draw a filled rectangle.
.   @param lineType Type of the line. See #LineTypes
.   @param shift Number of fractional bits in the point coordinates.



rectangle(img, rec, color[, thickness[, lineType[, shift]]]) -> img
.   @overload
.   
.   use `rec` parameter as alternative specification of the drawn rectangle: `r.tl() and
.   r.br()-Point(1,1)` are opposite corners
Type:      builtin_function

### What we are going to do ?
- Read and show video stream , capture images
- Detect Faces and show bounding box using (haarcascade).
- Flatten the largest face image and save in a numpy array- for that I need to convert it into gray scale image.
- Repeat the above for multiple people to generate training data

In [3]:
import cv2
import numpy as np


### Initializing the Camera:

In [4]:
capture = cv2.VideoCapture(0) # default position as only single stream 

### Face Detection - loading of a file

In [5]:
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")


### Flatten the largest face image and save in a numpy array

In [6]:
import cv2
import numpy as np
from tempfile import TemporaryFile 

capture = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip = 0 
# storing every 10th face
face_list = []
dataset_path = 'face_detection_folder/' # creating a dataset which stores the data at this path
file_name = input("Enter the name of person :")   

while True:
    return_value, frame = capture.read()
    
    gray_frame = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY) # to for face detection

   
    if return_value  == False:
        continue
    
#face detection
    
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
# it is a list of  faces where faces are  tuple  in form of (x,y,w,h) 
    
    
    

# Now we will sort the faces, using area of the face which is w*h,

    faces = sorted(faces, key = lambda f: f[2]*f[3])
    
#pick the largest face according to the area
    for face in faces[-1:] :#in descending order for largest face
        x, y, w, h = face
        cv2.rectangle(frame, (x,y), (x+w, y+ h), (0, 255, 255), 10)
        


#Extract (crop out the required face) : also called Region of interest
        
        offset = 10 


# pixels in each dirfection of rectangle adding
        
        face_section = frame[(y - offset):(y + h + offset), (x -offset): (x + w + offset)]
        face_section = cv2.resize(face_section, (100, 100)) 
        # into a resized frame
        skip = skip + 1 # when we do this we arecapturing the 10th frame each time 
        if (skip % 10 == 0):
     
            face_list.append(face_section)
            print(len(face_list))
            
            
        cv2.imshow("Face Section", face_section)
    cv2.imshow("Frame", frame)
    
        
# stopping the stream
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break



# convert our face_list into a numpy array
face_array = np.array(face_list)
face_array = face_array.reshape((face_array.shape[0], -1))
print(face_array.shape)


#saving the data into file system
np.save(dataset_path + file_name +".npy", face_array)
# this is how we can save any file
print("Data successfully saved at " + dataset_path +" "+ file_name +".npy")

capture.release()
cv2.destroyAllWindows()
    
#output is no.  of frames

Enter the name of person : Om


2024-09-29 00:35:38.661 python[5116:179184] +[IMKClient subclass]: chose IMKClient_Legacy
2024-09-29 00:35:38.661 python[5116:179184] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


1
2
3
4
5
6
7
(7, 30000)
Data successfully saved at face_detection_folder/ Om.npy


### Recognising Faces using some classification algorithm like 
- logistic
- KNN
- SVM
### Process:
- Read a video stream using opencv
- extract the faces out of it for testing
- load the training data (numpy arrays of all the persons)
  1. x-values are stored in numpy arrays
  2. y-values we need to assign  for each person
- Use KNN to find the prediction of face(int)
- map the prediction id to name of the user
- Display the predictions on the screen - bounding box and name


In [7]:
import cv2
import numpy as np
import os

In [8]:
def distance(v1, v2):
    #eucledian
    return np.sqrt(((v1 - v2)**2).sum())

In [9]:
def knn(train,test, k = 5):
    dist = []
    for i in range(train.shape[0]):
        #get the vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        #Compute the distance from test point.
    d = distance(test, ix)
    dist.append([d, iy])
    # sort based on distance and get top k
    dk = sorted(dist, key = lambda x : x[0][:k])
    #retrieve only the labels 
    labels = np.array(dk)[:, -1]
    # Get frequencies of each label
    output = np.unique(labels, return_counts = True)
    #Find max frequency and corresponding label 
    index = np.argmax(output[1])
    return output[0][index]
    

In [10]:
import cv2

In [11]:
capture = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip = 0 
dataset_path = 'face_detection_folder/'
# storing every 10th face
face_list = [] # training data
label = []  #y values of data
class_id = 0 # labels for the given file,it will start from zero and than increment 
names = {} # ,mapping btw id and  name



#data Preparation - where fx is some random file
for fx in os.listdir(dataset_path): # iterrating overmy own directory
    if fx.endswith('.npy'):
        names[class_id] = fx[:-4]#labelling the file name and remove .npy
        print("Loaded" + fx)# 4 chracters-->> .npy
        data_item = np.load(dataset_path + fx)
        face_list.append(data_item)

# create labels for the class:
# there are several files in our folder , each file have some faces of person and 
# it will bascially have 10 rows ,a some no. features (for example)
# for all the x values we created the array of size 10
# so, our target will be of shape 10 and will have a class id of 0 , as it is the first
# first person , class id for second person weill be one and so on....
# so for the first person we will get an array of zeros with 10 rows , similarly
# second will have an multiplied each element with 1 and so on...
# for each training point in each file we are computing one label using class_id
# so basically , my data is files and my label is the outpur array recieved from
# each file using class_id*np.ones((data_item.shape[0],))
# So,there are multiple data sets[x1,x2,x3,#files] and multiples labels[[y1], [y2] ]
# both of them are lsit , we will concatinate to make it a single list   
    target = class_id*np.ones((data_item.shape[0],))
    class_id += 1
    label.append(target)
face_dataset = np.concatenate(face_list, axis = 0) # all the x1,x2 ,x3 lists 
face_label   = np.concatenate(label, axis = 0).reshape((-1, 1))
# all the corresponding targets
print(face_dataset.shape)
print(face_label.shape)

trainset = np.concatenate((face_dataset, face_label), axis = 1)
print(trainset.shape)

#testing 
while True :
    return_value , frame = capture.read()
    if return_value == False:
        continue
    faces = face_cascade.detectMultiscale(frame, 1.3, 5)

    for face in faces:
        x,y,w,h = face
        offset = 10
        face_section = frame[(y - offset):(y + h + offset), (x -offset): (x + w + offset)]
        face_section = cv2.resize(face_section, (100, 100)) 

        output = knn(trainset, face_section.flatten())

        #display on the screen the name and  rectangle around it
        prediction_name = name[int(out)]
        cv2.putText(frame, prediction_name, (x, y- 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0,0))
        cv2.rectangle(frame, (x,y), (x+w, y+ h ), (0, 255, 255), 2)
            
        
    cv2.imshow("Faces", frame)
    
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
capture.release()
capture.destroyAllWindows()




# KNN accepts a x data and y data together as 1 argument 
# So, we will concatenate x and y



Loadedomkar12.npy
Loadedomkar13.npy
LoadedOm.npy
(39, 30000)
(46, 1)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 39 and the array at index 1 has size 46

### Stopping the stream:
- must be written in the program , otherwise videostream wont open

In [13]:
key_pressed = cv2.waitKey(1) & 0xFF
if key_pressed == ord('q'):
    break
capture.release()
capture.destroyAllWindows()

SyntaxError: 'break' outside loop (3750933369.py, line 3)

### video stream basic program to write

In [2]:
import cv2
import numpy as np

In [2]:
capture = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
while True:
    return_value, frame = capture.read()
    

    if return_value == False :
        continue
    cv2.imshow("Frame", frame)
    key_pressed_user =cv2.waitKey(1) & 0xFF 
    if key_pressed_user == ord('q'): 
        break
    

capture.release()
cv2.destroyAllWindows()


2024-09-29 00:36:18.702 python[5155:180220] +[IMKClient subclass]: chose IMKClient_Legacy
2024-09-29 00:36:18.702 python[5155:180220] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


### Programme to write face detection

In [3]:
capture = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
while True:
    return_value, frame = capture.read()
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if return_value == False :
        continue
    
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)
    print(faces)



    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 255, 0), 10)

    cv2.imshow("Video Frame", frame)
    key_pressed_user =cv2.waitKey(1) & 0xFF 
    
    if key_pressed_user == ord('q'): 
        break

capture.release()
cv2.destroyAllWindows()


[[750 335 671 671]]
[[752 339 671 671]]
[[752 339 671 671]]
[[750 341 666 666]]
[[764 327 674 674]]
[[765 329 674 674]]
[[765 329 674 674]]
[[768 329 678 678]]
[[762 323 685 685]]


2024-09-29 00:36:48.523 python[5167:180561] +[IMKClient subclass]: chose IMKClient_Legacy
2024-09-29 00:36:48.523 python[5167:180561] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


[[767 326 681 681]]
[[763 318 691 691]]
[[768 316 691 691]]
[[772 321 682 682]]
[[770 320 682 682]]
[[775 327 681 681]]
[[767 326 681 681]]
[[767 326 681 681]]
[[771 319 678 678]]
[[776 326 666 666]]
[[777 328 669 669]]
[[768 326 678 678]]
[[773 328 669 669]]
[[780 333 663 663]]
[[780 329 663 663]]
[[782 329 666 666]]
[[778 313 676 676]]
[[769 294 692 692]]
[[774 297 692 692]]
[[778 290 692 692]]
[[779 288 696 696]]
[[773 289 696 696]]
[[768 284 705 705]]
[[773 292 696 696]]
[[776 301 688 688]]
[[778 301 692 692]]
[[787 312 678 678]]
[[782 310 686 686]]
[[776 308 691 691]]
[[778 308 686 686]]
[[794 325 666 666]]
[[784 317 681 681]]
[[782 316 682 682]]
[[784 317 681 681]]
[[787 317 674 674]]
[[782 308 684 684]]
[[785 306 686 686]]
[[784 310 681 681]]
[[778 297 696 696]]
[[778 297 696 696]]
[[782 303 691 691]]
[[781 303 687 687]]
[[791 298 682 682]]
[[787 299 684 684]]
[[778 277 715 715]]
[[791 293 687 687]]
[[784 292 692 692]]
[[790 292 686 686]]
[[782 285 696 696]]
[[788 288 691 691]]


### Printing the frame , cropping of face 

In [4]:
import cv2
import numpy as np
capture = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
skip = 0 
face_data = []
dataset_path = 'face_detection_folder' 
    
while True:
    return_value, frame = capture.read()
    gray_frame = cv2.cvtColor(frame , cv2.COLOR_BGR2GRAY)

   
    if return_value  == False:
        continue
    faces = face_cascade.detectMultiScale(frame, 1.3, 5)
    faces = sorted(faces, key = lambda f: f[2]*f[3])
    

    for face in faces[-1:] :
        x, y, w, h = face
        cv2.rectangle(frame, (x,y), (x+w, y+ h), (0, 255, 255), 10)
        offset = 10 
        face_section = frame[(y - offset):(y + h + offset), (x -offset): (x + w + offset)]
        face_section = cv2.resize(face_section, (100, 100)) 
        
        skip = skip + 1 
        if(skip % 10 == 0):
            face_data.append(face_section)
            print(len(face_data))
            
        cv2.imshow("Face Section", face_section)
    cv2.imshow("Frame", frame)
    key_pressed = cv2.waitKey(1) & 0xFF
    if key_pressed == ord('q'):
        break
capture.release()
cv2.destroyAllWindows()

    

1
2
3
4
5
6
7
8
9


### Learn to Save File:

In [6]:
dataset_path = 'face_detection_folder' 
file_name = input("Enter the name of person :")
np.save(dataset_path + file_name +".npy", face_array)# save the data
print("Data successfully saved at " + dataset_path +" "+ file_name +".npy")

Enter the name of person : om


NameError: name 'face_array' is not defined